In [1]:
# df_adjOHLCV has data only for NYSE trading days, no weekend data
# df_adjOHLCV includes data for weekends when BTC trades
# read symbols in file to list syms_in_file
# download OHLCV data for symbols in syms_in_file
# drop symbols with all NaN in OHLCV columns from df
# rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# drop weekend data by reindex to date index of index_symbol
# pickled df_adjOHLCV
# pickled df_adjOHLCV
# pickled symbols_df_adjOHLCV
# create df_symbols_close, sort df by symbols
# pickled df_symbols_close

![alt text](image_url)

In [2]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'vg_symbols_4chars_max_clean.csv'  # symbols text file
filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

In [3]:
# function to adjust OHLCV
# https://github.com/ranaroussi/yfinance/issues/687
def auto_adjust(data):
    df = data.copy()
    ratio = df["Close"] / df["Adj Close"]
    df["Adj Open"] = df["Open"] / ratio
    df["Adj High"] = df["High"] / ratio
    df["Adj Low"] = df["Low"] / ratio

    df.drop(
        ["Open", "High", "Low", "Close"],
        axis=1, inplace=True)

    df.rename(columns={
        "Adj Open": "Open", "Adj High": "High",
        "Adj Low": "Low", "Adj Close": "Close"
    }, inplace=True)

    # df = df[["Open", "High", "Low", "Close", "Volume"]]
    return df[["Open", "High", "Low", "Close", "Volume"]]

def close_vs_Yahoo (symbols, df):
  for symbol in symbols:
    s = df.iloc[-250]
    
    sDate = s.name.strftime('%Y-%m-%d')
    sClose = s[symbol].Close

    df_sym = yf.Ticker(symbol).history(period='2y')
    # 10:45PM got Adj Close
    yhClose = df_sym.loc[sDate]['Close']

    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_err = f'{symbol}  %_dif_Close > .0001'
      # raise SystemExit(msg_err)
      print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
    if symbol == symbols[-1]:
      msg_done = f"No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close "
      print(msg_done)


In [4]:
# Stop if last date in df_adjOHLCV is the same as Yahoo download
index_symbol = "XOM"  
df_XOM = yf.download(index_symbol)
download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
print(f'download_last_date: {download_last_date}')
df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
df_adjOHLCV_last_date = df.index[-1].strftime('%Y-%m-%d')
print(f'df_adjOHLCV_last_date: {df_adjOHLCV_last_date}')

##########  Allow Repeated Download on the Same Day  ###########################################
# if download_last_date == df_adjOHLCV_last_date:
#   msg_stop = f"df_adjOHLCV is up-to-date,\nlast date from df_adjOHLCV is {df_adjOHLCV_last_date},\nlast date from Yahoo's {index_symbol} download is {download_last_date}"
#   raise SystemExit(msg_stop )
######################################################

[*********************100%***********************]  1 of 1 completed
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
download_last_date: 2023-11-14
df_adjOHLCV_last_date: 2023-11-13


In [5]:
# verify df test_symbols' close against Yahoo
# even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
test_symbols = ['A', 'LMT', 'SPY']
_df_list=[]
_df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)
_df_list.append(_df)
_df = pd.concat(_df_list, axis=1)
# get unique symbols from column name
_l_symbols = list(set(_df.columns.get_level_values(0)))
_l_symbols.sort()

_df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  _df1 = auto_adjust(_df[symbol])
  _df_adj_list.append(_df1)

_df_adj = pd.concat(_df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
_df_adj.columns = _col_names

close_vs_Yahoo(test_symbols, _df_adj)

[*********************100%***********************]  3 of 3 completed
symbol:     A   Date: 2022-11-16   df_Close:  146.08792     Yahoo_Close:  146.08791   %_dif_Close: 0.00001
symbol:   LMT   Date: 2022-11-16   df_Close:  454.33517     Yahoo_Close:  454.33517   %_dif_Close: 0.00000
symbol:   SPY   Date: 2022-11-16   df_Close:  389.32123     Yahoo_Close:  389.32123   %_dif_Close: 0.00000
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [6]:
# read symbols in file to a list and convert to chunks
#  each chunk must have more than 1 symbol, otherwise, symbol won't appear as column name
symbols_in_file = read_symbols_file(filename_symbols)
print(f'symbols in file: {len(symbols_in_file)}')
symbols_add = ['SPY', 'VGT', 'FTEC', 'QQQ', 'VWO', 'VTV', 'BNDX', 'USO', 'JJC', 'BCI', 'NG', 'GBTC', 'BTC-USD', 'ETH-USD', 'SGOV', 'GLD']
print(f'symbols added:   {len(symbols_add)}')
symbols = symbols_in_file + symbols_add
# https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python
symbols = list(set(symbols))  # keep unique symbols
symbols.sort()  # sort unique symbols in alphabetical order
print(f'unique symbols:  {len(symbols)}')
symbols_chunks = chunked_list(symbols, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]
for chunk in symbols_chunks:
  if len(chunk) == 1:
    raise SystemExit(f'ERROR, Must be more than 1 symbol in each chunk\nsymbols in chunk: {chunk}')

symbols in file: 1578
symbols added:   16
unique symbols:  1587


In [7]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(symbols_chunks):
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
  symbols_download_err.append(list(shared._ERRORS.keys()))
  # print(f'symbols_download_err: {symbols_download_err}')
  print(f'\nsymbols_download_err: {symbols_download_err}')  
  df_list.append(df)
  # pause between download
  if i < len(symbols_chunks) - 1 :  # skip pause after last download
    print(f'downloaded symbols from chuck {i}, sleep start')
    # sleep 78(18m 25s)
    time.sleep(78)
    print(f'downloaded symbols from chuck {i}, sleep ends')
  else:
    print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

[*********************100%***********************]  400 of 400 completed

7 Failed downloads:
- AJRD: No data found, symbol may be delisted
- BKI: No data found, symbol may be delisted
- DCP: No data found, symbol may be delisted
- AAWW: No data found, symbol may be delisted
- ATCO: No data found, symbol may be delisted
- ABC: No data found, symbol may be delisted
- ABB: No data found, symbol may be delisted

symbols_download_err: [['AJRD', 'BKI', 'DCP', 'AAWW', 'ATCO', 'ABC', 'ABB']]
downloaded symbols from chuck 0, sleep start
downloaded symbols from chuck 0, sleep ends
[*********************100%***********************]  400 of 400 completed

6 Failed downloads:
- HHC: No data found, symbol may be delisted
- IAA: No data found, symbol may be delisted
- FRC: No data found, symbol may be delisted
- FISV: No data found, symbol may be delisted
- KDNY: No data found, symbol may be delisted
- ISEE: No data found, symbol may be delisted

symbols_download_err: [['AJRD', 'BKI', 'DCP', 'AAWW',

In [8]:
# flatten symbols_download_err which is a list-of-lists
l_symbols_err = [val for sublist in symbols_download_err for val in sublist]
pickle_dump(l_symbols_err, path_data_dump, 'l_symbols_err')
l_symbols_err

['AJRD',
 'BKI',
 'DCP',
 'AAWW',
 'ATCO',
 'ABC',
 'ABB',
 'HHC',
 'IAA',
 'FRC',
 'FISV',
 'KDNY',
 'ISEE',
 'MGI',
 'PKI',
 'PDCE',
 'LSI',
 'RE',
 'NEX',
 'NUVA',
 'TA',
 'UNVR',
 'SJR',
 'WWE',
 'SCHN',
 'ROCC']

In [9]:
# if l_symbols_err:  # re-download symbols with download error 
#   symbols_download_err = []
#   # took 24 minutes to download 1917 symbols without error caused by Yahoo
#   # for i, symbols in enumerate(symbols_chunks):
#   for i, symbols in enumerate(l_symbols_err):
#     df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
#     symbols_download_err.append(list(shared._ERRORS.keys()))
#     # print(f'symbols_download_err: {symbols_download_err}')
#     print(f'\nsymbols_download_err: {symbols_download_err}')  
#     df_list.append(df)
#     # pause between download
#     if i < len(symbols_chunks) - 1 :  # skip pause after last download
#       print(f'downloaded symbols from chuck {i}, sleep start')
#       # sleep 78(18m 25s)
#       time.sleep(78)
#       print(f'downloaded symbols from chuck {i}, sleep ends')
#     else:
#       print(f'downloaded symbols from all chucks')

#   print(f'symbols_download_err: {symbols_download_err}')

In [10]:
df = pd.concat(df_list, axis=1)
if l_symbols_err:  # if list is not empty, drop symbols with errors
  print(f'l_symbols_err is not empty: {l_symbols_err}')  
  df.drop(l_symbols_err, axis=1, level=0, inplace=True)  # drop symbols with errors

l_symbols = list(set(df.columns.get_level_values(0)))  # get unique symbols
l_symbols.sort()
df = df[l_symbols]  # sort columns by list
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
pickle_dump(df, path_data_dump, filename_pickled_df_OHLCVA_downloaded)  # save OHLCA data

l_symbols_err is not empty: ['AJRD', 'BKI', 'DCP', 'AAWW', 'ATCO', 'ABC', 'ABB', 'HHC', 'IAA', 'FRC', 'FISV', 'KDNY', 'ISEE', 'MGI', 'PKI', 'PDCE', 'LSI', 'RE', 'NEX', 'NUVA', 'TA', 'UNVR', 'SJR', 'WWE', 'SCHN', 'ROCC']
Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [11]:
# adjust OHLC
df_adj_list=[]
for symbol in l_symbols:
  _df = auto_adjust(df[symbol])
  df_adj_list.append(_df)

df_adj = pd.concat(df_adj_list, axis=1)
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
col_names = pd.MultiIndex.from_product([l_symbols, cols])  # create multilevel column names
df_adj.columns = col_names  # reindex columns

print(f"Full path to pickled df_adjOHLCV: {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adj, path_data_dump, filename_pickled_df_adjOHLCV)  # save adjusted OHLCV data

Full path to pickled df_adjOHLCV: C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [12]:
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV)

In [13]:
# drop symbols with all NaN in OHLCV columns from df
df_adjOHLCV, symbols_OHLCV, symbols_dropped = drop_symbols_all_NaN(df_adjOHLCV, verbose)
print(f'symbols with all NaN dropped from df_adjOHLCV: {symbols_dropped}')

Symbol's OHLCV are all NaN: []
symbols with all NaN dropped from df_adjOHLCV: []


In [14]:
# rename columns OHLCV *ONLY AFTER* dropping symbols with all NaN from df,
#   symbols with all NaN has an added AdjClose column and will cause errors  
#  rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
#  .remove_unused_levels() prevents ValueError
#   e.g ValueError: On level 1, code max (5) >= length of level (5). NOTE: this index is in an inconsistent state
# The error may be caused by removing symbols from the dataframe with all NaN in OHLCV columns
df_adjOHLCV.columns = df_adjOHLCV.columns.remove_unused_levels()
# # set_levels reorders df columns in alphabetical order, so the list of column names also needs to be in alphabetical order
# df_adjOHLCV.columns = df_adjOHLCV.columns.set_levels(['close', 'high', 'low', 'open', 'volume'], level=1)

In [15]:
# drop weekend data by re-indexing to date-index of index_symbol
myNaN = float('nan')
# use Exxon's date as proxy for NYSE trading dates
df_adjOHLCV = df_adjOHLCV.reindex(df_XOM.index, fill_value=myNaN)

In [16]:
# pickle df_adjOHLCV and symbols
# print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
# pickle_dump(df_adjOHLCV_downloaded, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adjOHLCV, path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
pickle_dump(symbols_OHLCV, path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [17]:
symbols_OHLCV = list(set([i[0] for i in list(df_adjOHLCV)]))
df_symbols_close = df_adjOHLCV.xs('Close', level=1, axis=1)
pickle_dump(df_symbols_close, path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

In [18]:
# retrieve pickled files

print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA_downloaded = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
symbols_df = pickle_load(path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [19]:
print(f'df_OHLCVA_downloaded.info():\n{df_OHLCVA_downloaded.info()}\n')
print(f'df_adjOHLCV.info():\n{df_adjOHLCV.info()}\n')
print(f'df_close.info():\n{df_close.info()}\n')
print(f'len(symbols_df):\n{len(symbols_df)}\n')

<class 'pandas.core.frame.DataFrame'>
Index: 16614 entries, 1962-01-02 00:00:00 to 2023-11-14 00:00:00
Columns: 9366 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(9366)
memory usage: 1.2+ GB
df_OHLCVA_downloaded.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15575 entries, 1962-01-02 to 2023-11-14
Columns: 7805 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(7805)
memory usage: 927.6 MB
df_adjOHLCV.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15575 entries, 1962-01-02 to 2023-11-14
Columns: 1561 entries, A to ZWS
dtypes: float64(1561)
memory usage: 185.6 MB
df_close.info():
None

len(symbols_df):
1561



In [20]:
count_symbols = len(symbols_df)
count_df_close_cols = len(df_close.columns)
count_cols_df_OHLCVA_downloaded = len(df_OHLCVA_downloaded.columns)
count_cols_df_adjOHLCV = len(df_adjOHLCV.columns)
if count_symbols != count_df_close_cols:
  raise SystemExit(f'symbol count: {count_symbols} does not equal df_close column count: {count_df_close_cols}')
if count_symbols * 6 != count_cols_df_OHLCVA_downloaded:
  raise SystemExit(f'6 x symbol count: {6 * count_symbols} does not equal df_OHLCVA_downloaded column count: {count_cols_df_OHLCVA_downloaded}')
if count_symbols * 5 != count_cols_df_adjOHLCV:
  raise SystemExit(f'5 x symbol count: {5 * count_symbols} does not equal df_adjOHLCV column count: {count_cols_df_adjOHLCV}')
print(f'No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count')


No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count


In [21]:
print(f'count_symbols: {count_symbols}')
print(f'count_df_close_cols: {count_df_close_cols}')
print(f'count_cols_df_OHLCVA_downloaded: {count_cols_df_OHLCVA_downloaded}')
print(f'count_cols_df_adjOHLCV: {count_cols_df_adjOHLCV}')

count_symbols: 1561
count_df_close_cols: 1561
count_cols_df_OHLCVA_downloaded: 9366
count_cols_df_adjOHLCV: 7805


In [22]:
close_vs_Yahoo(test_symbols, df_adjOHLCV)

symbol:     A   Date: 2022-11-16   df_Close:  146.08792     Yahoo_Close:  146.08791   %_dif_Close: 0.00001
symbol:   LMT   Date: 2022-11-16   df_Close:  454.33521     Yahoo_Close:  454.33517   %_dif_Close: 0.00001
symbol:   SPY   Date: 2022-11-16   df_Close:  389.32123     Yahoo_Close:  389.32126   %_dif_Close: 0.00001
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [23]:
_sym = 'NOC'
s_start, s_end = -252, -248
df_adjOHLCV[_sym].iloc[s_start:s_end]
df_OHLCVA_downloaded[_sym].iloc[s_start:s_end]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-03-08 00:00:00,472.000000,475.630005,460.920013,465.250000,461.238678,1089000.0
2023-03-09 00:00:00,469.630005,469.630005,458.200012,459.040009,455.082245,834900.0
2023-03-10 00:00:00,458.529999,463.190002,457.190002,459.779999,455.815826,772200.0
2023-03-11 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_adjOHLCV.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-11-08  110.889999  111.570000  108.459999  109.389999  1495000.0   
2023-11-09  110.120003  110.120003  107.540001  107.739998  1481100.0   
2023-11-10  107.720001  108.480003  104.089996  108.470001  1847800.0   
2023-11-13  108.129997  108.269997  106.959999  107.459999  1200300.0   
2023-11-14  109.550003  112.290001  109.459999  111.610001  1971475.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-11-08  25.120001  25.660000  24.965000  25.530001  3671400.0  ...   
2023-11-09  25.809999  25.870001  24.410000  24.580000  4078500.0  ...   
2023-11-10  24.700001  24.889999  24.280001  24.639999  3039400.0  ...   
2023-11-13  24.490000  25.025000  24.299999  24.600000  2839700.0  ...   
2023-11-14  25.440001  26.565001  25.360001  26.170000  5419367.0  ...   

                 ZUMZ                                               ZWS  \
                 Open       High    Low      Close    Volume       Open   
Date                                                                      
2023-11-08  16.500000  16.540001  15.83  15.910000  201200.0  28.379999   
2023-11-09  16.120001  16.219999  15.17  15.340000  240300.0  28.420000   
2023-11-10  15.370000  15.760000  14.99  15.740000  259500.0  28.280001   
2023-11-13  15.670000  15.940000  15.50  15.730000  115900.0  28.700001   
2023-11-14  16.270000  17.766001  16.27  17.709999  248992.0  29.360001   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-11-08  28.379999  28.059999  28.299999  1163100.0  
2023-11-09  28.490000  28.049999  28.049999  1468500.0  
2023-11-10  28.860001  28.059999  28.799999  1413600.0  
2023-11-13  28.830000  28.389999  28.600000  1027700.0  
2023-11-14  30.030001  29.360001  29.950001  1213668.0  

[5 rows x 7805 columns]

In [25]:
syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
_df = df_close[syms_def]
_df.tail()


,NOC,HII,AVAV,LMT,CW,HEI
Date,,,,,,
2023-11-08,460.480011,228.520004,123.459999,444.660004,204.720001,160.470001
2023-11-09,461.089996,228.139999,120.379997,442.459991,205.830002,162.520004
2023-11-10,463.269989,229.929993,121.589996,444.869995,207.479996,165.330002
2023-11-13,464.380005,233.360001,125.900002,445.029999,213.369995,166.899994
2023-11-14,466.829987,235.919998,129.210007,444.220001,214.660004,170.539993


In [26]:
syms_LNG = ['LNG', 'CHK', 'GLNG']
_df = df_close[syms_LNG]
_df.tail()

,LNG,CHK,GLNG
Date,,,
2023-11-08,170.220001,81.400002,21.990000
2023-11-09,170.669998,80.489998,21.809999
2023-11-10,171.619995,79.089996,22.010000
2023-11-13,170.919998,80.010002,22.230000
2023-11-14,172.220001,81.540001,22.559999


In [27]:
_df = df_close[symbols_add]
_df.tail()

,SPY,VGT,FTEC,QQQ,VWO,VTV,BNDX,USO,JJC,BCI,NG,GBTC,BTC-USD,ETH-USD,SGOV,GLD
Date,,,,,,,,,,,,,,,,
2023-11-08,437.250000,436.839996,130.000000,372.940002,39.459999,137.309998,48.480000,70.190002,NaN,20.760000,3.60,27.799999,35655.277344,1889.322388,100.379997,180.770004
2023-11-09,433.839996,434.220001,129.309998,370.070007,39.110001,136.190002,48.290001,70.250000,NaN,20.680000,3.57,29.090000,36693.125000,2120.561035,100.430000,181.490005
2023-11-10,440.609985,445.690002,132.550003,378.390015,39.380001,137.630005,48.330002,71.790001,NaN,20.610001,3.58,29.973000,37313.968750,2078.289795,100.430000,179.509995
2023-11-13,440.190002,443.459991,131.889999,377.209991,39.450001,137.550003,48.320000,72.919998,NaN,20.969999,3.65,29.059999,36502.355469,2055.265381,100.440002,180.509995
2023-11-14,448.730011,454.190002,134.940002,385.329987,40.369999,140.020004,48.689999,72.690002,NaN,20.980000,3.97,27.420000,35557.460938,1986.400513,100.480003,182.119995


In [28]:
# syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
s_start, s_end = -252, -247
_df = df_close[test_symbols].iloc[s_start:s_end]
_df

,A,LMT,SPY
Date,,,
2022-11-14,145.353119,450.242462,388.996338
2022-11-15,146.961746,455.319427,392.314117
2022-11-16,146.087921,454.335205,389.321228
2022-11-17,143.426727,460.698456,388.129944
2022-11-18,145.164459,464.645081,389.892212
